In [ ]:
# default_exp preprocess.label_data

# Preparing the list of customers in the scope 

> API details.

In [2]:
#hide
from nbdev.showdoc import *

In [4]:
#export
import os
import pandas as pd
from sample_project import config
from sample_project.helper import get_spark_session
from fastcore.utils import store_attr
import numpy as np
from sample_project.preprocess import prepare_transaction_data

In [12]:
#hide
import warnings
warnings.filterwarnings("ignore")

In [13]:
#export
class Label_Data:
    
    '''
    This class is built to label main dataset which is created with prepare_transaction_data module. Below steps are followed:
    1. Adjust error in end_date field in account information dataset 
    2. Label first individual accounts by taking reference date into consideration
    3. Consider activeness of all accounts belong to the customers and label them as churner if there is no active account anymore
        
    '''
    
    def __init__(self, trnx_dataset = None, account_info_dataset=None, reference_date = 981231):
        
        '''
        Args:
            trnx_dataset (Pandas DataFrame): The transaction dataset which has at least these fields: "account_id","client_id"
            account_info_dataset (Pandas DataFrame): The account info dataset which has at least these fields: "account_id", "start_date", "end_date"
            reference_date (integer): The date to be considered in activeness check of accounts

        Return:
            labelled_data (pandas DataFrame)
        '''
        
        store_attr()
        
        if trnx_dataset == None:
            trnx_data_creator = prepare_transaction_data.Create_Data(trnx_dataset=None, disp_dataset=None, client_dataset=None, loan_dataset=None, loan_amnt_thrsh=0, district_cnt_thrsh = 0)
            self.trnx_dataset = trnx_data_creator()
        
        if account_info_dataset == None:
            self.account_info_dataset = pd.read_csv(config.DATA_DIR + config.CSV_ACCOUNT_INFO, index_col=[0])
    
    def __call__(self):
        
        self._fix_error_in_account_info_dataset()
        self._label_accounts()
        self._label_customers()
        
        return self._merge_label_with_main_data()
        
    def _fix_error_in_account_info_dataset(self): 
        
        self.account_info_dataset = self.account_info_dataset.merge(
                                                                    self.trnx_dataset.groupby("account_id",as_index=False).date.max().rename(columns={'date':'Last_Trnx_Date'}),
                                                                    on="account_id", how="left"
                                                                   )
        
        self.account_info_dataset["end_date"] = np.where(self.account_info_dataset["end_date"] < self.account_info_dataset["Last_Trnx_Date"], self.account_info_dataset["Last_Trnx_Date"],self.account_info_dataset["end_date"])
        return
    
    def _label_accounts(self): 
        
        self.account_info_dataset["active_or_not"] = np.where(self.account_info_dataset["end_date"] <= self.reference_date, 0 ,np.where(self.account_info_dataset["start_date"]<self.reference_date,1,0))
        return
     
    def _label_customers(self):
        
        self.labelled_cust_data = (self.trnx_dataset[["account_id","client_id"]]
                               .drop_duplicates()
                               .merge(self.account_info_dataset[["account_id","active_or_not"]],on="account_id",how="left")
                              )
        
        self.labelled_cust_data = self.labelled_cust_data.groupby("client_id",as_index=False).active_or_not.sum()
        self.labelled_cust_data["churn_or_not"] = np.where(self.labelled_cust_data["active_or_not"] == 0, 1 ,0)
        return
    
    def _merge_label_with_main_data(self):
        
        return self.trnx_dataset.merge(self.labelled_cust_data[["client_id","active_or_not"]],on="client_id",how="left")

In [14]:
#hide
labeller = Label_Data( reference_date = 981231)
labelled_trnx_data = labeller()
labelled_trnx_data

,trans_id,account_id,date,type,operation,amount,balance,client_id,district_id,Total_Loan_Amount,active_or_not
0,1548749,5270,930113,"""PRIJEM""","""VKLAD""",800.0,800.0,6367,44,79608.0,1
1,1548750,5270,930114,"""PRIJEM""","""PREVOD Z UCTU""",44749.0,45549.0,6367,44,79608.0,1
2,3393738,11265,930114,"""PRIJEM""","""VKLAD""",1000.0,1000.0,13845,15,52788.0,1
3,3122924,10364,930117,"""PRIJEM""","""VKLAD""",1100.0,1100.0,12754,55,21924.0,1
4,3122924,10364,930117,"""PRIJEM""","""VKLAD""",1100.0,1100.0,12755,55,21924.0,1
...,...,...,...,...,...,...,...,...,...,...,...
233622,3626751,2910,981231,"""PRIJEM""","""""",187.2,42249.6,3517,68,437460.0,1
233623,3627577,2933,981231,"""PRIJEM""","""""",132.0,31046.4,3542,10,272520.0,1
233624,3627577,2933,981231,"""PRIJEM""","""""",132.0,31046.4,3543,10,272520.0,1
233625,3626847,2912,981231,"""PRIJEM""","""""",271.6,58694.2,3519,74,380160.0,1
